In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
from pandas import DataFrame, read_csv

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 6000)


In [28]:
data = pd.read_csv("train_set.csv", encoding="utf-8")


/home/andrey/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
for i in ['atm_address', 'city', 'atm_address_lon', 'customer_id', 
          'pos_address', 'terminal_id', 'transaction_date']:
    if i in data.columns:
        data[i]=data[i].astype('object')

In [30]:
for i in ['amount', 'atm_address_lat','atm_address_lon','currency', 'home_add_lat', 
          'home_add_lon', 'mcc', 'pos_adress_lat','pos_adress_lon','work_add_lat','work_add_lon']:
    if i in data.columns:
        data[i]=data[i].astype('float64')

In [31]:
data['city'] = data['city'].str.upper()
data['country'] = data['country'].str.upper()
data['pos_address'] = data['pos_address'].str.upper()

data['city'] = pd.Series(data['city']).str.replace('М','M')
data['city'] = pd.Series(data['city']).str.replace('К','K')
data['city'] = pd.Series(data['city']).str.replace('А','A')
data['city'] = pd.Series(data['city']).str.replace('С','C')
data['city'] = pd.Series(data['city']).str.replace('О','O')
data['city'] = pd.Series(data['city']).str.replace('Е','E')
data['city'] = pd.Series(data['city']).str.replace('Т','T')
data['city'] = pd.Series(data['city']).str.replace('Р','P')
data['city'] = pd.Series(data['city']).str.replace('В','B')
data['city'] = pd.Series(data['city']).str.replace(' ','')


In [33]:
#Обработка Санкт-Петербурга
x=0

for i in range(len(data)):
    if str(data['city'][i]).find('PETER') > -1 and str(data['city'][i]) != 'ST.PETERSBURG' and str(data['city'][i]) != 'PETERGOF' and str(data['city'][i]) != 'GPETERGOF' and str(data['city'][i]) != 'PETERGOFG':
        print(data['city'][i])
        data.at[data['city'] == data['city'][i], 'city'] = 'ST.PETERSBURG'
        x=x+1 
print(x)     

#Обработка Москвы
x=0
for i in range(len(data)):
    if str(data['city'][i]).find('MOS') > -1 and str(data['city'][i]) != 'MOSCOW' and str(data['city'][i]) != 'NOVOMOSKOVSK' and str(data['city'][i]) != 'MOSKOVSKIY' and str(data['city'][i]) != 'MOSTOVSKOY' and str(data['city'][i]) != 'P.MOSTOVSKOY' and str(data['city'][i]) != 'ZAMOSTE':
        print(data['city'][i])
        data.at[data['city'] == data['city'][i], 'city'] = 'MOSCOW'
        x=x+1
print(x)

STPETERSBURG
SANKT-PETERBU
SAINT-PETERSB
SPETERSBURG
SAINTPETERSB
ST.-PETERSBUR
S-PETER
STPETERBURG
SANKT-PETERSB
SANKT-PETERS
ST-PETERBURG
STPETERSBUR
PETERBURG
ST-PETERBUR
S-PETERBURG
ST-PETER.
S.PETERBURG
SANKTPETERBU
SANTPETERBUR
SAINT-PETERBU
ST.PETERSBUR
SANQTPETERBU
SANKT-PETER
SPETERBURG
S.PETERSBURG
GSANKT-PETER
SANK-PETERBUR
ST.PETERBURG
S-PETERSBURG
SANKT-PETERUB
C-PETERBURG
SNPETERBURG
SANKT-PETERBO
S.-PETERBURG
ST/PETERSBURG
SANPETERBURG
ST-PETERB.
ST-PETERURG
ST-PETER
SPETERSRURG
ST-PETERSBURG
ST.-PETERBURG
SANKTPETERSB
S.PETERBURG,
SAINTPETERBU
45
MOSKVA
GMOSKVA
MOSKVAREGION
MOSKOW
MOSCOWREG.
MOSKOVSKAYAO
MOSKOVSKIIYR
MOSKVA,G.KRA
MOSCOWREGION
MOSKOVSK.OBL.
MOSVA
MOSALSK
MOSCOWOBL.
MOSKVA,CEV.
DOLGIYMOST
MOSCOWSKAYAO
G.MOSKVA
MOSKOVSKAYA
MOSKVA,SEVER
MOSKOWREGION
MOSKVAG.MOSK
MOSKVA,P.VNU
MOSKVAP.VNUK
MOSKVA,P.DE
MOSCHELKOVO
MOSKVA,G.ZE
KAMENNOMOSTSK
MOSKOVSKIY,22
MOSKVA,P.SOSE
POSELOKMOSTO
MOSTOVSKIIYR
MOSKVAG.VOS
MOSKVA,G.MO
MOSKVA,POS.
MOSKVA,G.MOS
MOSKOVSKII
MOSKVAPO

In [35]:
pd.set_option('display.max_rows', 1000)

data.at[data['city'] == '--NEZADAN-', 'city'] = 'OTHER'
data.at[data['city'] == ',', 'city'] = 'OTHER'
data.at[data['city'] == 'CPB', 'city'] = 'ST.PETERSBURG'
data.at[data['city'] == 'SPB', 'city'] = 'ST.PETERSBURG'
data.at[data['city'] == '******', 'city'] = 'OTHER'
data.at[data['city'] == 'G.SANKT-PETE', 'city'] = 'ST.PETERSBURG'
data.at[data['city'] == '.', 'city'] = 'OTHER'
data.at[data['city'] == 'LIPETSK', 'city'] = 'LIPECK'
data.at[data['city'] == '', 'city'] = 'LIPECK'
data.at[data['city'] == 'M', 'city'] = 'MOSCOW'
data['city'] = pd.Series(data['city']).str.replace('>','')
data['city'] = pd.Series(data['city']).str.replace(',',' ')

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 6000)


In [44]:
import geocoder

data_city = data['city'].unique()
df_data_city = pd.DataFrame(data = data_city, columns = ['city'])

data_cordinate = pd.DataFrame(columns=['lat_city_country','lng_city_country'])
for x in range(len(data_city)):
    g = geocoder.google(df_data_city['city'][x])
    data_cordinate.loc[x] = [g.lat, g.lng]
    
data_cordinate.to_csv('data_cordinate.csv')

KeyboardInterrupt: 

In [ ]:
data_city = data['city'].unique()
df_data_city = pd.DataFrame(data = data_city, columns = ['city'])

c = 0
for i in range(len(df_data_city)):
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  ====  ',df_data_city['city'][i])
    for x in range(len(df_data_city)):
        if x == i and x <= 616:
            print(x)
            c=1
        else:
            c=0
        w1 = set(str(df_data_city['city'][i]))
        w2 = set(str(df_data_city['city'][x+c]))
        c=0
        if (len(w1.intersection(w2))/len(w2))*100 > 97.0:
                 data.at[data['city'] == data['city'][x+c], 'city'] = df_data_city['city'][i]

In [ ]:
data.at[data['country'] == 'RU', 'country'] = 'RUS'
data.at[data['country'] == 'DE', 'country'] = 'DEU'
data.at[data['country'] == 'IT', 'country'] = 'ITA'
data.at[data['country'] == 'BY', 'country'] = 'BLR'
data.at[data['country'] == ' ZZ"', 'country'] = 'OTHER'

data.loc[data['country'].value_counts()[data['country']].values < 47, 
         'country'] = 'OTHER'


In [ ]:
data['currency'].astype('category')

In [ ]:
for i in range(len(data)):
    dt = data['transaction_date'][i]
    year, month, day = (int(x) for x in dt.split('-'))    
    ans = datetime.date(year, month, day)
    data['transaction_date'][i] = ans.strftime("%A")

In [ ]:
data['currency'].fillna(data['currency'][1], inplace=True)
data['amount'].fillna(data['amount'].median(), inplace=True)
data['transaction_date'].fillna('2017-06-09', inplace=True)
data['terminal_id'].fillna('9663c3a3f047f284926a166405f30856', inplace=True)

In [ ]:
x = 1996

map_1 = folium.Map(location=[data['pos_adress_lat'][x], data['pos_adress_lon'][x]],
                   zoom_start=12,
                   tiles='Stamen Terrain')
folium.Marker([data['pos_adress_lat'][x], data['pos_adress_lon'][x]],
              popup='Mt. Hood Meadows',
              icon=folium.Icon(icon='cloud')
             ).add_to(map_1)
folium.Marker([data['home_add_lat'][x], data['home_add_lon'][x]],
              popup='Timberline Lodge',
              icon=folium.Icon(color='green')
             ).add_to(map_1)
folium.Marker([data['work_add_lat'][x], data['work_add_lon'][x]],
              popup='Some Other Location',
              icon=folium.Icon(color='red',icon='info-sign')
              ).add_to(map_1)
map_1

In [45]:
data_cordinate

,lat_city_country,lng_city_country
0,59.934280,30.335099
1,NaN,NaN
2,NaN,NaN
3,55.894055,37.443949
4,NaN,NaN
5,55.947064,37.499276
6,55.947064,37.499276
7,56.333382,36.730447
8,55.626012,37.849191
9,53.337199,91.933470


array(['ST.PETERSBURG', 'NOVOSIBIRSK', 'MOSCOW', ..., 'USTKAMENOGORS',
       'G.ARHANGELSK', 'MALYEDERBETY'], dtype=object)